In [1]:
from utilities.utils_clean import *
from utilities.test_dataset_utils import *

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\amoha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amoha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:

#download_patents_pto(start_year=2015, end_year=2015, kind='grant',download_path="data")
#unzip_files("data","patent_grants_2015")

In [ ]:
# # Example usage
# test_dataset = create_test_dataset_from_freilich(
#     year=2015,  # The year you want to analyze
#     freilich_data_path="../data/Freilich.Data.Compressed.xlsb",  # Path to your Freilich dataset
#     path_to_all_xmls_for_chosen_year="../data/patent_grants_2015"  # Directory containing XML files
# )

# # Optional: Check the results
# print(f"Number of patents extracted: {len(test_dataset)}")
# print("Sample document numbers:", list(test_dataset.keys())[:5])

In [11]:
# test_data = load_from_pickle("../data/test_dataset_2015.pkl")

# df_test_data = pd.DataFrame(test_data,index=["xml"],).T.reset_index()
# df_test_data.columns = ["patentnumber","xml"] 


# df = read_xlsb_file()
df[df.issueyear == 2018][["patentnumber","prophetic","nonprophetic","issueyear"]]
# df[df.issueyear == 2018].to_csv("freilichdataet_2018.csv")
# df = pd.read_csv("freilichdataet_2018.csv")
# df["patentnumber"] = df["patentnumber"].astype(str).transform(lambda x: x.replace(".0", ""))
# df["patentnumber"] = df["patentnumber"].apply(remove_leadiong_zeros)
# df.head()

,patentnumber,prophetic,nonprophetic,issueyear


In [12]:
df.issueyear.unique()

array([1970., 1974., 1976., 1977., 1978., 1979., 1980., 1981., 1982.,
       1983., 1984., 1985., 1986., 1987., 1988., 1989., 1990., 1991.,
       1992., 1993., 1994., 1995., 1996., 1997., 1998., 1999., 2000.,
       2001., 2002., 2003., 2004., 2005., 2006., 2007., 2008., 2009.,
       2010., 2011., 2012., 2013., 2014., 2015., 2016., 2017.])

In [6]:
merged = df_test_data.merge(df, on="patentnumber", how="left")
print(len(merged))
cols = ["patentnumber","xml","prophetic","nonprophetic","allprophetic","someprophetic"]
merged[cols].head()

25081


,patentnumber,xml,prophetic,nonprophetic,allprophetic,someprophetic
0,RE045323,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",1.0,12.0,0.0,1.0
1,RE045324,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",0.0,8.0,0.0,0.0
2,RE045325,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",0.0,8.0,0.0,0.0
3,8925349,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",26.0,0.0,1.0,0.0
4,8925551,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",6.0,5.0,0.0,1.0


In [10]:
import pprint as pp
pp.pprint(heading[0].find_next_siblings())


[<heading id="h-0005" level="1">S. General Synthesis Procedures for Examples</heading>,
 <p id="p-0083" num="0440">The examples of the present invention are synthesized according to the following general scheme:</p>,
 <p id="h-0006" num="0000">General Synthesis Scheme:</p>,
 <p id="p-0084" num="0441"><chemistry id="CHEM-US-00066" num="00066">
<img alt="embedded image" file="USRE045323-20150106-C00066.TIF" he="19.13mm" id="EMI-C00066" img-content="chem" img-format="tif" wi="73.32mm"/>
</chemistry>
<br/>
where “building block” and “reaction partner” are defined for the procedures A-G below in which A, R<sup>1</sup>, R<sup>2</sup>, R<sup>3 </sup>and R<sup>4 </sup>are defined as above. In case a building block or reaction partner is not commercially available, its synthesis is described below, commencing from a commercially available staring material, and where necessary, involving intermediate(s), “I”.
</p>,
 <p id="p-0085" num="0442">The term “linker” is used to mean a chemical moiety wh

In [18]:
find_doc_number(merged.xml[0])

['RE045323']

In [ ]:

from lxml import etree
import io

# Get the XML string
xml_str = merged[merged.patentnumber == highest_difference_patent].xml.values[0]

# Parse with BeautifulSoup instead of lxml
# This makes it easier to work with siblings

# Get all elements to process as siblings

# Extract examples
examples = extract_examples_start_w_word(siblings)

In [47]:
from bs4 import BeautifulSoup


num_of_paterns = 1000
mae = 0
highest_difference = 0
found_heading = 0
not_found_heading = 0
gib = 0
short = 0
mostdifss = []
doc_w_exp = {}

for row in merged[cols][:num_of_paterns].iterrows():
    xml = row[1]["xml"]
    if len(xml)>2000:
        s_tags = re.findall(r'<s\d+>.*?</s\d+>', xml)
        if len(s_tags) > 0:
            #print(f"Patent {row[1]['patentnumber']} is gibberish")
            gib+=1
        else:
            heading = extract_experiments_w_heading(xml)
            janetsnumexamples = row[1]["prophetic"] + row[1]["nonprophetic"]

            if heading:
                found_heading += 1
                examples = extract_examples_start_w_word_test(heading[0].find_next_siblings())
                if len(examples)==0:
                    soup = BeautifulSoup(xml, 'xml')
                    siblings = soup.find_all(['heading', 'p'])
                    examples = extract_examples_start_w_word_test(siblings)
                numexamples = len(examples)
            else:
                not_found_heading += 1
                soup = BeautifulSoup(xml, 'xml')
                siblings = soup.find_all(['heading', 'p'])
                examples = extract_examples_start_w_word_test(siblings)
                numexamples = len(examples)



            difference = abs(numexamples-janetsnumexamples)
            mae += difference
            if difference > 0:
                mostdifss.append([difference, row[1]["patentnumber"]])
            if difference > highest_difference:
                highest_difference = difference
                highest_difference_patent = row[1]["patentnumber"]
            if len(examples)>0:
                doc_w_exp[row[1]["patentnumber"]] = examples

    else:
        short+=1
        #print(f"skipping {row[1]['patentnumber']}, patent is too short")
        
        
print(f"found_heading: {found_heading}, not_found_heading: {not_found_heading}, gibberish: {gib},too_short: {short}")
print(f"{num_of_paterns} patents MAE: {mae/num_of_paterns}, total_error: {mae}, highest_difference: {highest_difference}, highest_difference_patent: {highest_difference_patent}")

found_heading: 534, not_found_heading: 272, gibberish: 170,too_short: 24
1000 patents MAE: 0.33, total_error: 330.0, highest_difference: 56.0, highest_difference_patent: 8933072


In [53]:
799/806

0.9913151364764268

In [50]:
534 + 272

806

In [62]:
print(f"Number of patents with difference > 0: {len(mostdifss)}")
mostdifss

Number of patents with difference > 0: 104


[[4.0, '8926548'],
 [1.0, '8926595'],
 [1.0, '8926732'],
 [1.0, '8926824'],
 [6.0, '8926862'],
 [4.0, '8926864'],
 [1.0, '8926872'],
 [1.0, '8926955'],
 [11.0, '8926966'],
 [7.0, '8926979'],
 [1.0, '8927015'],
 [1.0, '8927023'],
 [5.0, '8927036'],
 [2.0, '8927098'],
 [1.0, '8927150'],
 [2.0, '8927157'],
 [1.0, '8927236'],
 [4.0, '8927254'],
 [1.0, '8927429'],
 [1.0, '8927469'],
 [1.0, '8927479'],
 [1.0, '8927536'],
 [1.0, '8927541'],
 [1.0, '8927546'],
 [2.0, '8927568'],
 [4.0, '8927583'],
 [6.0, '8927588'],
 [3.0, '8927596'],
 [1.0, '8927617'],
 [1.0, '8927619'],
 [2.0, '8927631'],
 [1.0, '8927642'],
 [1.0, '8927647'],
 [1.0, '8927678'],
 [1.0, '8927681'],
 [2.0, '8927683'],
 [1.0, '8927698'],
 [7.0, '8927710'],
 [15.0, '8927721'],
 [1.0, '8927725'],
 [1.0, '8927730'],
 [1.0, '8927738'],
 [1.0, '8927746'],
 [1.0, '8927750'],
 [1.0, '8927777'],
 [1.0, '8927781'],
 [1.0, '8927801'],
 [9.0, '8927811'],
 [1.0, '8932406'],
 [3.0, '8932470'],
 [4.0, '8932480'],
 [2.0, '8932491'],
 [2.0, '89

In [71]:
s_tags = re.findall(r'<s\d+>.*?</s\d+>', xml)

In [68]:
xml = merged[merged.patentnumber == "8926966"].xml.values[0]
soup = BeautifulSoup(xml, 'xml')
siblings = soup.find_all(['heading', 'p'])
extract_examples_start_w_word_test(soup)

[]

In [65]:
merged[merged.patentnumber == "8926966"][cols]

,patentnumber,xml,prophetic,nonprophetic,allprophetic,someprophetic
78,8926966,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",0.0,11.0,0.0,0.0


In [ ]:
import numpy as np
final = dic_to_dic_w_tense_test(doc_w_exp,threshold=0)
df_final = pd.DataFrame(final).T.reset_index()
df_final.columns = ["patentnumber","past","present","Unknown"]
df_final["patentnumber"] = df_final["patentnumber"].apply(remove_leadiong_zeros)
df_check = df_final.merge(merged,on= "patentnumber",how="left")[["patentnumber","present","past","Unknown","prophetic","nonprophetic","allprophetic","someprophetic"]]
#df_check["past"] = df_check["past"] + df_check["Unknown"]
df_check["Total_Extracted"] = df_check["past"] + df_check["present"]  #+ df_check["Unknown"]
df_check["Total_Freilich"] = df_check["prophetic"] + df_check["nonprophetic"]
df_check["prophetic_error"] = np.sqrt((df_check["prophetic"] - df_check["present"])**2)
df_check["nonprophetic_error"] = np.sqrt((df_check["nonprophetic"] - df_check["past"])**2)
df_check["Total_Mean_error"] = np.sqrt((df_check["Total_Freilich"] - df_check["Total_Extracted"])**2)
df_check["Sum_error"] = df_check["prophetic_error"] + df_check["nonprophetic_error"] + df_check["Total_Mean_error"]
print(f"Number of exact matches: {len(df_check[(df_check.prophetic_error ==0) & (df_check.nonprophetic_error ==0) & (df_check.Total_Mean_error ==0)])} out of {len(df_check)} , Percentage: {len(df_check[(df_check.prophetic_error ==0) & (df_check.nonprophetic_error ==0) & (df_check.Total_Mean_error ==0)])/len(df_check)*100}")
print(f"Number of exact num of patent extracted: {len(df_check[(df_check.Total_Extracted == df_check.Total_Freilich)])} out of {len(df_check)}")
print(f"Avg Total error: {df_check['Total_Mean_error'].mean()}, num of corrects:  {df_check[df_check['Total_Mean_error'] == 0].shape[0]}")
print(f"Avg Total prophetic error: {df_check['prophetic_error'].mean()}, num of corrects: {df_check[df_check['prophetic_error'] == 0].shape[0]}")
print(f"Avg Total nonprophetic error: {df_check['nonprophetic_error'].mean()}, num of corrects: {df_check[df_check['nonprophetic_error'] == 0].shape[0]}")
print(f"{df_check['Sum_error'].sum()}")
print(f"Sum of number of Unknowns: {df_check['Unknown'].sum()}")
print(f"Number of patets with experiments extracted {len(df_check)}")
df_check.sort_values("Sum_error",ascending=False).head(20)

Number of exact matches: 386 out of 799 , Percentage: 48.310387984981226
Number of exact num of patent extracted: 681 out of 799
Avg Total error: 1.1739674593241551, num of corrects:  681
Avg Total prophetic error: 1.4217772215269087, num of corrects: 440
Avg Total nonprophetic error: 2.382978723404255, num of corrects: 401
3978.0
Sum of number of Unknowns: 670
Number of patets with experiments extracted 799


,patentnumber,present,past,Unknown,prophetic,nonprophetic,allprophetic,someprophetic,Total_Extracted,Total_Freilich,prophetic_error,nonprophetic_error,Total_Mean_error,Sum_error
289,8927710,2,220,349,0.0,564.0,0.0,0.0,222,564.0,2.0,344.0,342.0,688.0
207,8927555,1,11,92,0.0,104.0,0.0,0.0,12,104.0,1.0,93.0,92.0,186.0
197,8927536,0,187,68,0.0,254.0,0.0,0.0,187,254.0,0.0,67.0,67.0,134.0
558,8933072,2,63,0,0.0,9.0,0.0,0.0,65,9.0,2.0,54.0,56.0,112.0
202,8927546,51,377,0,0.0,428.0,0.0,0.0,428,428.0,51.0,51.0,0.0,102.0
209,8927557,646,52,25,685.0,38.0,0.0,1.0,698,723.0,39.0,14.0,25.0,78.0
321,8927757,0,38,0,37.0,1.0,0.0,1.0,38,38.0,37.0,37.0,0.0,74.0
193,8927531,0,36,0,36.0,0.0,1.0,0.0,36,36.0,36.0,36.0,0.0,72.0
565,8933085,35,153,1,5.0,184.0,0.0,1.0,188,189.0,30.0,31.0,1.0,62.0
581,8933104,44,69,0,18.0,100.0,0.0,1.0,113,118.0,26.0,31.0,5.0,62.0


In [ ]:
# patent 9102614 injanets says 4 examples but there are 161
# patent 9102599 in janets says 1 ,theres more than 1 actually 
# Some xml files are made with error : e.g. these dont have much information : skipping 9102601, 9102662,9102692
# these have gibberish: 9102724,9102705,9102727,9102628

In [ ]:
def process_siblings_test(xml_siblings):
    examples = []

    # Find all matching headings directly from xml_siblings
    example_headings = [
        tag
        for tag in xml_siblings
        if tag.name == "heading"
        and any(
            keyword in tag.text.strip().lower().replace(" ", "")
            for keyword in ["example", "experiment", "test"]
        )
        # and not any(
        #     excluded in tag.text.strip().lower().replace(" ", "")
        #     for excluded in ["reference", "preparation"]
        # )
    ]

    for heading in example_headings:
        current_content = []
        idx = xml_siblings.index(heading)

        # Get title from next heading if available
        title = ""
        if idx + 1 < len(xml_siblings) and xml_siblings[idx + 1].name == "heading":
            title = xml_siblings[idx + 1].text.strip()

        # Collect content until next example heading
        i = idx + 1
        while i < len(xml_siblings):
            if (
                xml_siblings[i].name == "heading"
                and any(
                    keyword in xml_siblings[i].text.strip().lower().replace(" ", "")
                    for keyword in ["example", "experiment", "test"]
                )
                # and not any(
                #     excluded in xml_siblings[i].text.strip().lower().replace(" ", "")
                #     for excluded in ["reference", "preparation"]
                # )
            ):
                break
            if xml_siblings[i].name == "p":
                current_content.append(xml_siblings[i].text.strip())
            i += 1

        examples.append(
            {"number": heading.text.strip(), "title": title, "content": current_content}
        )

    return examples if examples else None





In [21]:
heading = extract_experiments_w_heading(merged.xml[0])
test = process_siblings_test(heading[0].find_next_siblings())

In [29]:
combined_dict = {i: dic for i, dic in enumerate(test, 0)}
combined_dict.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56])

In [ ]:

extract_examples_start_w_word(heading[0].find_next_siblings())

[{'number': 'Examples 1-131',
  'title': 'Amide Formation Method A',
  'content': ['',
   'To a stirred solution of building block A (1 eq) in DMF, NMP, DCM or chloroform at room temperature is added DIPEA or TEA (2-8 eq), HATU or TBTU (1-4 eq) and HOBt (0-2 eq). After 20-45 min, amine reaction partner (1.2-2 eq) is added and the mixture stirred for an additional 40-120 min. Water and TFA is then added and the mixture is separated via reversed phase HPLC to provide the desired amides. The reaction mixture can be filtered over basic ALOX or extracted with DCM after addition of aqueous NaHCO3 solution followed by concentrating the organic fractions in vacuum before the HPLC purification.',
   '',
   'Building block A1 (60 mg) was dissolved in NMP (0.5 ml) and DIPEA (0.15 ml). HATU (120 mg) was added and the solution was stirred for 45 min at rt. 3-(1-methyl-1H-tetrazol-5-yl)aniline (34 mg) was added and the mixture was stirred for a further 1 h. A few drops of water and TFA were added an

In [ ]:
res = {}
for dic in test:
    res.update(dic)
res

{'number': 'Examples of Pharmaceutical Formulations',
 'title': '',
 'content': ['A)\nTablets\nper tablet\n\n\n\n\n\n\n\n\n\n\nactive substance 1\n100 mg\n\n\n\n\nlactose\n140 mg\n\n\n\n\nmaize starch\n240 mg\n\n\n\n\npolyvinylpyrrolidone\n\u200215 mg\n\n\n\n\nmagnesium stearate\n\u20035 mg\n\n\n\n\n\n500 mg',
  'The finely ground active substance, lactose and some of the maize starch are mixed together. The mixture is screened, then moistened with a solution of polyvinylpyrrolidone in water, kneaded, wet granulated and dried. The granules, the remaining maize starch and the magnesium stearate are screened and mixed together. The mixture is pressed into tablets of suitable shape and size.',
  'B)\nTablets\nper tablet\n\n\n\n\n\n\n\n\n\n\nactive substance 1\n80 mg\n\n\n\n\nlactose\n55 mg\n\n\n\n\nmaize starch\n190 mg\u2002\n\n\n\n\nmicrocrystalline cellulose\n35 mg\n\n\n\n\npolyvinylpyrrolidone\n15 mg\n\n\n\n\nsodium carboxymethyl starch\n23 mg\n\n\n\n\nmagnesium stearate\n\u20022 mg\n\

In [119]:

doc_w_exp = {}
i = 0
length_threshold = 50
for i,xml in enumerate(merged["xml"]):
    i += 1
    if i % 1000 == 0:
        print(f"{i}/{len(merged.xml)} so far found {len(doc_w_exp)} docs with experiments")
    heading = extract_experiments_w_heading(xml)
    if heading:
        extracted_examples = []
        if len(heading) >1:
            print(f"has more than 1 Examples heading , {i}")
            # for example in heading:
            #     extracted_ex_start_w_word = process_siblings(example.find_next_siblings())
            #     if extracted_ex_start_w_word and len(extracted_ex_start_w_word) > 0:
            #         extracted_examples.append(extracted_ex_start_w_word)
            #     else:
            #         extracted_ex_w_word = extract_examples_start_w_word(example.find_next_siblings())
            #         if extracted_ex_w_word:
            #             if len(extracted_ex_w_word) > 0:
            #                 extracted_examples.append(extracted_ex_w_word)
            #         else:
            #             num_dot_examples = extract_num_dot_examples(str(heading[0].find_next_siblings()))
            #             if len(num_dot_examples) > 0:
            #                 extracted_examples.append(num_dot_examples)
        elif len(heading) == 1:
            example_start_w = process_siblings(heading[0].find_next_siblings())
            if example_start_w:
                if len(example_start_w[0]["content"]) == 0:
                    extracted_ex_w_word = extract_examples_start_w_word(heading[0].find_next_siblings())
                    try:
                        if isinstance(extracted_ex_w_word, list):
                            for ex in extracted_ex_w_word:
                                if len(ex["content"]) > length_threshold:
                                    extracted_examples.append(ex)
                            extracted_examples.append(extracted_ex_w_word)
                        else:
                            if len(extracted_ex_w_word["content"]) > length_threshold:
                                extracted_examples.append(extracted_ex_w_word)
                    except:
                        if len(extracted_ex_w_word[0]["content"]) > length_threshold:
                            extracted_examples.append(extracted_ex_w_word)
                else:
                    extracted_examples.append(example_start_w)
            else:
                extracted_ex_w_word = extract_examples_start_w_word(heading[0].find_next_siblings())
                if isinstance(extracted_ex_w_word, list):
                    for ex in extracted_ex_w_word:
                        if len(ex["content"]) > length_threshold:
                            extracted_examples.append(ex)
                    extracted_examples.append(extracted_ex_w_word)
                else:
                    try:
                        if len(extracted_ex_w_word["content"]) > length_threshold:
                            extracted_examples.append(extracted_ex_w_word)
                    except:
                        num_dot_examples = extract_num_dot_examples(str(heading[0].find_next_siblings()))
                        if isinstance(num_dot_examples, list):
                            for ex in num_dot_examples:
                                if len(ex["content"]) > length_threshold:
                                    extracted_examples.append(ex)
                            extracted_examples.append(num_dot_examples)
                        else:
                            if len(num_dot_examples["content"]) > length_threshold:
                                extracted_examples.append(num_dot_examples)
                    
                    
    else:
        extracted_ex_w_word = extract_examples_w_word(xml)
        try:
            if isinstance(extracted_ex_w_word, list):
                for ex in extracted_ex_w_word:
                    if len(ex["content"]) > length_threshold:
                        extracted_examples.append(ex)
                extracted_examples.append(extracted_ex_w_word)
            else:
                if len(extracted_ex_w_word["content"]) > length_threshold:
                    extracted_examples.append(extracted_ex_w_word)
        except:
            example_start_w = process_siblings(heading[0].find_next_siblings())
            if isinstance(example_start_w, list):
                for ex in example_start_w:
                    if len(ex["content"]) > length_threshold:
                        extracted_examples.append(ex)
                extracted_examples.append(example_start_w)
            else:
                if len(example_start_w["content"]) > length_threshold:
                    extracted_examples.append(example_start_w)

            # else:
            #     num_dot_examples = extract_num_dot_examples(str(heading[0].find_next_siblings()))
            #     extracted_examples.append(num_dot_examples)
    
        

has more than 1 Examples heading , 349
has more than 1 Examples heading , 388
has more than 1 Examples heading , 689
has more than 1 Examples heading , 768
has more than 1 Examples heading , 781
has more than 1 Examples heading , 814
1000/25081 so far found 507 docs with experiments
has more than 1 Examples heading , 1054
has more than 1 Examples heading , 1417
has more than 1 Examples heading , 1475


TypeError: list indices must be integers or slices, not str

In [6]:
def extract_examples_from_heading(heading):
    extracted_examples = []
    example_start_w = process_siblings(heading.find_next_siblings())
    if example_start_w:
        if not example_start_w[0]["content"]:
            extracted_ex_w_word = extract_examples_start_w_word(heading.find_next_siblings())
            if extracted_ex_w_word:
                if isinstance(extracted_ex_w_word, list):
                    if extracted_ex_w_word and len(extracted_ex_w_word[0]["content"])>0:
                        extracted_examples.append(extracted_ex_w_word)
                elif extracted_ex_w_word["content"]:
                    extracted_examples.append(extracted_ex_w_word)
        else:
            if len(example_start_w[0]["content"])>0:
                extracted_examples.append(example_start_w)
    else:
        extracted_ex_w_word = extract_examples_start_w_word(heading.find_next_siblings())
        if extracted_ex_w_word:
            if isinstance(extracted_ex_w_word, list):
                if extracted_ex_w_word and len(extracted_ex_w_word[0]["content"])>0:
                    extracted_examples.append(extracted_ex_w_word)
            elif extracted_ex_w_word["content"]:
                if len(extracted_ex_w_word["content"])>0:
                    extracted_examples.append(extracted_ex_w_word)
        else:
            num_dot_examples = extract_num_dot_examples(str(heading.find_next_siblings()))
            if num_dot_examples:
                extracted_examples.append(num_dot_examples)
    return extracted_examples

def extract_examples(xml):
    heading = extract_experiments_w_heading(xml)
    if heading:
        if len(heading) > 1:
            print(f"has more than 1 Examples heading, {i}")
        elif len(heading) == 1:
            return extract_examples_from_heading(heading[0])
        else:
            extracted_ex_w_word = extract_examples_w_word(xml)
            if extracted_ex_w_word:
                if isinstance(extracted_ex_w_word, list):
                    if extracted_ex_w_word and extracted_ex_w_word[0]["content"]:
                        return [extracted_ex_w_word]
                elif extracted_ex_w_word["content"]:
                    return [extracted_ex_w_word]
            else:
                example_start_w = process_siblings(heading[0].find_next_siblings())
                if example_start_w and example_start_w[0]["content"]:
                    return [example_start_w]
    else: 
        example_start_w = extract_examples_w_word(xml)
        if example_start_w:
            return [example_start_w]
        else:
            num_dot_examples = extract_num_dot_examples(xml)
            if num_dot_examples:
                return [num_dot_examples]
    return []

doc_w_exp = {}
for i, xml in enumerate(merged["xml"][:1000], start=1):
    if i % 1000 == 0:
        print(f"{i}/{len(merged.xml)} so far found {len(doc_w_exp)} docs with experiments")
    
    extracted_examples = extract_examples(xml)
    if extracted_examples:
        doc_w_exp[find_doc_number(xml)[0]] = extracted_examples

has more than 1 Examples heading, 349
has more than 1 Examples heading, 388
has more than 1 Examples heading, 689
has more than 1 Examples heading, 768
has more than 1 Examples heading, 781
has more than 1 Examples heading, 814
1000/25081 so far found 771 docs with experiments


In [73]:
def extract_examples_from_heading(heading):
    length_threshold = 30
    extracted_examples = []
    example_start_w = process_siblings(heading.find_next_siblings())
    if example_start_w:
        if not example_start_w[0]["content"]:
            extracted_ex_w_word = extract_examples_start_w_word(heading.find_next_siblings())
            if extracted_ex_w_word:
                if isinstance(extracted_ex_w_word, list):
                    extracted_ex_w_word = [ex for ex in extracted_ex_w_word if len(ex["content"])>length_threshold]
                    extracted_examples.append(extracted_ex_w_word)
                elif extracted_ex_w_word["content"]>length_threshold:
                    extracted_examples.append(extracted_ex_w_word)
        else:
            if isinstance(example_start_w, list):
                example_start_w = [ex for ex in example_start_w if len(ex["content"])>length_threshold]
                extracted_examples.append(example_start_w)
    else:
        extracted_ex_w_word = extract_examples_start_w_word(heading.find_next_siblings())
        if extracted_ex_w_word:
            if isinstance(extracted_ex_w_word, list):
                extracted_ex_w_word = [ex for ex in extracted_ex_w_word if len(ex["content"])>length_threshold]
                extracted_examples.append(extracted_ex_w_word)
            elif len(extracted_ex_w_word["content"])>length_threshold:
                extracted_examples.append(extracted_ex_w_word)
        else:
            num_dot_examples = extract_num_dot_examples(str(heading.find_next_siblings()))
            if isinstance(num_dot_examples, list):
                num_dot_examples = [ex for ex in num_dot_examples if len(ex["content"])>length_threshold]
                extracted_examples.append(num_dot_examples)
            elif isinstance(num_dot_examples, dict):
                num_dot_examples = [ex for ex in num_dot_examples.items() if len(ex["content"])>length_threshold]

                extracted_examples.append(num_dot_examples)
    return extracted_examples

def extract_examples(xml):
    length_threshold = 30
    heading = extract_experiments_w_heading(xml)
    if heading:
        if len(heading) > 1:
            print(f"has more than 1 Examples heading, {i}")
        elif len(heading) == 1:
            return extract_examples_from_heading(heading[0])
        else:
            pass
        # else:
        #     extracted_ex_w_word = extract_examples_w_word(xml)
        #     if extracted_ex_w_word:
        #         if isinstance(extracted_ex_w_word, list):
        #             extracted_ex_w_word = [ex for ex in extracted_ex_w_word if len(ex["content"])>length_threshold]
        #             return extracted_ex_w_word if extracted_ex_w_word
        #         elif extracted_ex_w_word["content"]>length_threshold:
        #             return extracted_ex_w_word if extracted_ex_w_word
        #     else:
        #         example_start_w = process_siblings(heading[0].find_next_siblings())
        #         if isinstance(example_start_w, list):
        #             example_start_w = [ex for ex in example_start_w if len(ex["content"])>length_threshold]
        #             return example_start_w if example_start_w
    else: 
        example_start_w = extract_examples_w_word(xml)
        if isinstance(example_start_w, list):
            example_start_w = [ex for ex in example_start_w if len(ex["content"])>length_threshold]
            return example_start_w
        else:
            num_dot_examples = extract_num_dot_examples(xml)
            if isinstance(num_dot_examples, list):
                num_dot_examples = [ex for ex in num_dot_examples if len(ex["content"])>length_threshold]
                return num_dot_examples

doc_w_exp = {}
for i, xml in enumerate(merged["xml"][:1000], start=1):
    if i % 100 == 0:
        print(f"{i}/{len(merged.xml)} so far found {len(doc_w_exp)} docs with experiments")
    
    extracted_examples = extract_examples(xml)
    if extracted_examples:
        doc_w_exp[find_doc_number(xml)[0]] = extracted_examples



100/25081 so far found 54 docs with experiments
200/25081 so far found 111 docs with experiments
300/25081 so far found 169 docs with experiments
has more than 1 Examples heading, 349
has more than 1 Examples heading, 388
400/25081 so far found 225 docs with experiments
500/25081 so far found 294 docs with experiments
600/25081 so far found 352 docs with experiments
has more than 1 Examples heading, 689
700/25081 so far found 395 docs with experiments
has more than 1 Examples heading, 768
has more than 1 Examples heading, 781
800/25081 so far found 463 docs with experiments
has more than 1 Examples heading, 814
900/25081 so far found 522 docs with experiments
1000/25081 so far found 575 docs with experiments


In [93]:

doc_w_exp = {}
for i, xml in enumerate(merged[:1999].xml.values, start=1):
    if i % 1000 == 0:
        print(f"{i}/{len(merged.xml)} so far found {len(doc_w_exp)} docs with experiments")
    
    extracted_examples = extract_examples(xml)
    if extracted_examples:
        doc_w_exp[find_doc_number(xml)[0]] = extracted_examples

has more than 1 Examples heading, 349
has more than 1 Examples heading, 388
has more than 1 Examples heading, 689
has more than 1 Examples heading, 768
has more than 1 Examples heading, 781
has more than 1 Examples heading, 814
1000/25081 so far found 769 docs with experiments
has more than 1 Examples heading, 1054
has more than 1 Examples heading, 1417
has more than 1 Examples heading, 1475
has more than 1 Examples heading, 1810


In [5]:
# save_as_pickle(doc_w_exp,"doc_w_exp_new_algorithm.pkl")
doc_w_exp = load_from_pickle("doc_w_exp_new_algorithm.pkl")

# save_as_json(doc_w_exp, "doc_w_exp_2015.json")
# doc_w_exp = read_json("../data/doc_w_exp_2015.json")

Loaded 18977 patents from doc_w_exp_new_algorithm.pkl


In [10]:
import re
import html
import unicodedata
from bs4 import BeautifulSoup

def clean_text_updated(text):
    """
    Clean text by removing HTML tags, special characters, extra spaces,
    and normalizing the content while keeping meaningful punctuation.

    Args:
        text (str): Input text to clean

    Returns:
        str: Cleaned text
    """
    if not isinstance(text, str):
        return ""

    # Decode HTML entities & remove HTML tags
    text = html.unescape(text)
    text = BeautifulSoup(text, "html.parser").get_text()

    # Normalize Unicode characters (e.g., é → e)
    text = unicodedata.normalize("NFKC", text)

    # Remove unwanted special characters but keep punctuation
    text = re.sub(r"[^\w\s.,!?'\-]", "", text)

    # Normalize spaces: remove multiple spaces, newlines, and tabs
    text = re.sub(r"\s+", " ", text).strip()

    return text

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

def check_tense_nltk_updated(sentence):
    words = word_tokenize(sentence)
    tagged = pos_tag(words)

    # Define tense categories
    past = {"VBD", "VBN"}
    present = {"VB", "VBG", "VBP", "VBZ"}
    future = {"MD"}

    tenses = {"past": 0, "present": 0, "future": 0}

    for i, (word, tag) in enumerate(tagged):
        # Count past tense words
        if tag in past:
            tenses["past"] += 1

        # Count present tense words
        elif tag in present:
            tenses["present"] += 1

        # Future tense handling
        elif tag in future:
            if word.lower() in {"will", "shall"}:
                # Ensure 'will' or 'shall' is followed by a verb (to confirm future tense)
                if i + 1 < len(tagged) and tagged[i + 1][1] in {"VB", "VBP"}:
                    tenses["future"] += 1

    return max(tenses, key=tenses.get) if max(tenses.values()) > 0 else "Unknown"



In [ ]:
import re
from utilities.nlp_processing import check_tense_nltk,clean_text

def tense_with_pattern(description):
    future_pattern = r'\b(?:will|would|shall|should)\b'
    past_pattern = r'\b(?:was|were|had|did)\b'

    future_matches = re.findall(future_pattern, description)
    past_matches = re.findall(past_pattern, description)
    if future_matches and past_matches:
        if len(future_matches) > len(past_matches):
            return "present"
        else:
            return "past"
    if future_pattern or past_pattern:
        if future_pattern:
            return "present"
        else:
            return "past"
    return "Unkown"


def dic_to_dic_w_tense(doc_w_exp,threshold = 50):
    dic = {}
    pattern = r'\(\d+\)\s*([A-Za-z0-9\-\(\)\{\},:;=\[\]\+\*\s\.\^\$\%]+(?:\.(?:sup|delta|Hz|NMR)[^\)]*)?)'
    pattern2 = r'\(\d+\)\s*(?:[A-Za-z]+\s*\d*\-?[A-Za-z]*[\(\{][^)]*[\)\}][^;]*|(?:\.\d*H\-?NMR[^;]*|\d+\.[a-zA-Z]*\d*))'


    for key, value in doc_w_exp.items():
        tense_counts = {"past": 0, "present": 0,"Unknown":0}
        
        if isinstance(value, list) and len(value) == 1 and len(value[0][0]["content"]) > threshold:
            desc = value[0][0]["title"] + "." + "".join(value[0][0]["content"])
            tense = check_tense_nltk_updated(clean_text(desc))
            if tense !=  "Unknown":
                tense_counts[tense] += 1
            else:
                matches = re.findall(pattern, desc)
                if matches:
                    tense_counts["past"] += 1
                else:
                    tense_counts["Unknown"] += 1
            dic[key] = tense_counts


        elif isinstance(value[0], list) and len(value[0]) > 1:
            for ls in value[0]:
                if len(ls["content"]) > threshold:
                    desc = "".join(ls["content"])
                    tense = check_tense_nltk_updated(clean_text(desc))
                    if tense !=  "Unknown":
                        tense_counts[tense] += 1
                    else:
                        matches = re.findall(pattern, desc)
                        if matches:
                            tense_counts["past"] += 1
                        else:
                            tense_counts["Unknown"] += 1
            dic[key] = tense_counts

        elif isinstance(value[0], dict):
            for ex, desc in value[0].items():
                if len(desc) > threshold:
                    tense = check_tense_nltk_updated(clean_text(desc))
                    if tense !=  "Unknown":
                        tense_counts[tense] += 1
                    else:
                        matches = re.findall(pattern, desc)
                        if matches:
                            tense_counts["past"] += 1
                        else:
                            tense_counts["Unknown"] += 1
            dic[key] = tense_counts
        else:
            print(type(value[0]))
            print(value[0])

    return dic

import re

def dic_to_dic_w_unknown_tense(doc_w_exp):
    dic = {}
    pattern = r'\(\d+\)\s*([A-Za-z0-9\-\(\)\{\},:;=\[\]\+\*\s\.\^\$\%]+(?:\.(?:sup|delta|Hz|NMR)[^\)]*)?)'
    pattern2 = r'\(\d+\)\s*(?:[A-Za-z]+\s*\d*\-?[A-Za-z]*[\(\{][^)]*[\)\}][^;]*|(?:\.\d*H\-?NMR[^;]*|\d+\.[a-zA-Z]*\d*))'

    for key, value in doc_w_exp.items():
        unknown_experiments = []  # Initialize a list to store the experiments classified as Unknown
        
        if isinstance(value[0], list) and len(value[0]) == 1:
            desc = value[0][0]["title"] + "." + "".join(value[0][0]["content"])
            tense = check_tense_nltk_updated(clean_text(desc))
            if tense == "Unknown":
                unknown_experiments.append(desc)

        elif isinstance(value[0], list) and len(value[0]) > 1:
            for ls in value[0]:
                desc = ls["title"] + "." + "".join(ls["content"])
                tense = check_tense_nltk_updated(clean_text(desc))
                if tense == "Unknown":
                    unknown_experiments.append(desc)

        elif isinstance(value[0], dict):
            for ex, description in value[0].items():
                tense = check_tense_nltk_updated(clean_text(description))
                if tense == "Unknown":
                    unknown_experiments.append(description)

        # Only add to the dictionary if there are "Unknown" experiments
        if unknown_experiments:
            dic[key] = unknown_experiments

    return dic


In [105]:
# 8946300 example 1-5
# 08946296 example 2 onwards 
# 08932484 examples 2 to 11


# 08933074:  
# Synthesis of 5,5-dimethyl-4-(pyridin-4-yl)-3-(4-(quinolin-2-ylmethoxy)phenyl) furan-2(5H)-one (Example 23)
# 5,5-dimethyl-4-(pyridin-4-yl)-3-(4-(quinolin-2-ylmethoxy)phenyl) furan-2(5H)-one (Example 23)

# 08945522:
# Example 2
# (69) 3-(3-(Triethoxysilyl)propylamino)phenyl benzoate and
# (140) Example 9
# (141) 4-(3-(Triethoxysilyl)propylamino)phenyl benzoate and
# (142) Example 10
# (143) 4-(Bis(3-(triethoxysilyl)propyl)amino)phenyl benzoate



SyntaxError: invalid syntax (1140748136.py, line 1)

In [9]:
final = dic_to_dic_w_unknown_tense(doc_w_exp)
print(len(final))
final

c:\ProgramData\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


145


{'RE045323': ['Examples 1-131.',
  'Example 116.',
  'Examples 132-191.',
  'Synthesis of Example 132.',
  'Examples 192-196.',
  'Example 192.',
  'Examples 197-208.',
  'Example 197.',
  'Examples 210-212.',
  'Example 210.',
  'Examples 213-216.',
  'Example 213.'],
 '08926732': ['Gas Permeability Experiments.'],
 '08926736': ['Separation Tests.', 'Cyclability Tests.'],
 '08926740': ['Production Example of Compound (16).'],
 '08926872': ['Example 1.'],
 '08926918': ['Example 1A (Comparative).'],
 '08926932': ['Example Pore Size Distribution.'],
 '08926944': ['Synthesis of (16S,20S)-10,18-dioxo-1-((4-((1,4,7,10-tetrakis(carboxymethyl)-1,4,7,10-tetraazacyclododecan-2-yl)methyl)phenyl)amino)-1-thioxo-2,11,17,19-tetraazadocosane-16,20,22-tricarboxylic acid (MIP-1512).Electrospray ionisation mass spectral analysis: [MS (ESI)]: (M+H)+=1013.',
  'Synthesis of (16S,20S)-10,18-dioxo-1-((4-((1,4,7,10-tetrakis(carboxymethyl)-1,4,7,10-tetraazacyclododecan-2-yl)methyl)phenyl)amino)-1-thioxo-2,11

In [34]:
print(f"Sum of number of Unknowns: {df_check['Unknown'].sum()}")

Sum of number of Unknowns: 693


In [33]:
for thresh in [0,5,10,15,20]:
    print(f"Threshold: {thresh}")
    final = dic_to_dic_w_tense(doc_w_exp,threshold=thresh)
    df_final = pd.DataFrame(final).T.reset_index()
    df_final.columns = ["patentnumber","past","present","Unknown"]
    df_final["patentnumber"] = df_final["patentnumber"].apply(remove_leadiong_zeros)
    df_check = df_final.merge(merged,on= "patentnumber",how="left")[["patentnumber","present","past","Unknown","prophetic","nonprophetic","allprophetic","someprophetic"]]
    #df_check["past"] = df_check["past"] + df_check["Unknown"]
    df_check["Total_Extracted"] = df_check["past"] + df_check["present"]  #+ df_check["Unknown"]
    df_check["Total_Freilich"] = df_check["prophetic"] + df_check["nonprophetic"]
    df_check["prophetic_error"] = np.sqrt((df_check["prophetic"] - df_check["present"])**2)
    df_check["nonprophetic_error"] = np.sqrt((df_check["nonprophetic"] - df_check["past"])**2)
    df_check["Total_Mean_error"] = np.sqrt((df_check["Total_Freilich"] - df_check["Total_Extracted"])**2)
    df_check["Sum_error"] = df_check["prophetic_error"] + df_check["nonprophetic_error"] + df_check["Total_Mean_error"]
    print(f"Number of exact matches: {len(df_check[(df_check.prophetic_error ==0) & (df_check.nonprophetic_error ==0) & (df_check.Total_Mean_error ==0)])} out of {len(df_check)} , Percentage: {len(df_check[(df_check.prophetic_error ==0) & (df_check.nonprophetic_error ==0) & (df_check.Total_Mean_error ==0)])/len(df_check)*100}")
    print(f"Number of exact num of patent extracted: {len(df_check[(df_check.Total_Extracted == df_check.Total_Freilich)])} out of {len(df_check)}")
    print(f"Avg Total error: {df_check['Total_Mean_error'].mean()}, num of corrects:  {df_check[df_check['Total_Mean_error'] == 0].shape[0]}")
    print(f"Avg Total prophetic error: {df_check['prophetic_error'].mean()}, num of corrects: {df_check[df_check['prophetic_error'] == 0].shape[0]}")
    print(f"Avg Total nonprophetic error: {df_check['nonprophetic_error'].mean()}, num of corrects: {df_check[df_check['nonprophetic_error'] == 0].shape[0]}")
    print(f"{df_check['Sum_error'].sum()}")
    print(f"Sum of number of Unknowns: {df_check['Unknown'].sum()}")
    df_check.sort_values("Total_Mean_error",ascending=False).head(20)

Threshold: 0


c:\ProgramData\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


Number of exact matches: 248 out of 772 , Percentage: 32.12435233160622
Number of exact num of patent extracted: 489 out of 772
Avg Total error: 3.0259067357512954, num of corrects:  489
Avg Total prophetic error: 1.5906735751295338, num of corrects: 394
Avg Total nonprophetic error: 3.911917098445596, num of corrects: 273
6584.0
Threshold: 5


KeyboardInterrupt: 

In [40]:
df_final = pd.DataFrame(final).T.reset_index()
df_final.columns = ["patentnumber","past","present","Unknown"]
df_final["patentnumber"] = df_final["patentnumber"].apply(remove_leadiong_zeros)
df_check = df_final.merge(merged,on= "patentnumber",how="left")[["patentnumber","present","past","Unknown","prophetic","nonprophetic","allprophetic","someprophetic"]]
#df_check["past"] = df_check["past"] + df_check["Unknown"]
df_check["Total_Extracted"] = df_check["past"] + df_check["present"]  #+ df_check["Unknown"]
df_check["Total_Freilich"] = df_check["prophetic"] + df_check["nonprophetic"]
df_check["prophetic_error"] = np.sqrt((df_check["prophetic"] - df_check["present"])**2)
df_check["nonprophetic_error"] = np.sqrt((df_check["nonprophetic"] - df_check["past"])**2)
df_check["Total_Mean_error"] = np.sqrt((df_check["Total_Freilich"] - df_check["Total_Extracted"])**2)
df_check["Sum_error"] = df_check["prophetic_error"] + df_check["nonprophetic_error"] + df_check["Total_Mean_error"]
print(f"Number of exact matches: {len(df_check[(df_check.prophetic_error ==0) & (df_check.nonprophetic_error ==0) & (df_check.Total_Mean_error ==0)])} out of {len(df_check)} , Percentage: {len(df_check[(df_check.prophetic_error ==0) & (df_check.nonprophetic_error ==0) & (df_check.Total_Mean_error ==0)])/len(df_check)*100}")
print(f"Number of exact num of patent extracted: {len(df_check[(df_check.Total_Extracted == df_check.Total_Freilich)])} out of {len(df_check)}")
print(f"Avg Total error: {df_check['Total_Mean_error'].mean()}, num of corrects:  {df_check[df_check['Total_Mean_error'] == 0].shape[0]}")
print(f"Avg Total prophetic error: {df_check['prophetic_error'].mean()}, num of corrects: {df_check[df_check['prophetic_error'] == 0].shape[0]}")
print(f"Avg Total nonprophetic error: {df_check['nonprophetic_error'].mean()}, num of corrects: {df_check[df_check['nonprophetic_error'] == 0].shape[0]}")
print(f"{df_check['Sum_error'].sum()}")
print(f"number of Unknown tensed examples: {df_check['Unknown'].sum()}")
print(f"Total of examples in freilich {df_check.Total_Freilich.sum()},Total examples extracted {df_check.Total_Extracted.sum()}")
df_check.sort_values("Total_Mean_error",ascending=False).head(20)

Number of exact matches: 248 out of 772 , Percentage: 32.12435233160622
Number of exact num of patent extracted: 489 out of 772
Avg Total error: 3.0259067357512954, num of corrects:  489
Avg Total prophetic error: 1.5906735751295338, num of corrects: 394
Avg Total nonprophetic error: 3.911917098445596, num of corrects: 273
6584.0
number of Unknown tensed examples: 693
Total of examples in freilich 11470.0,Total examples extracted 12514


,patentnumber,present,past,Unknown,prophetic,nonprophetic,allprophetic,someprophetic,Total_Extracted,Total_Freilich,prophetic_error,nonprophetic_error,Total_Mean_error,Sum_error
277,8927710,30,225,349,0.0,564.0,0.0,0.0,255,564.0,30.0,339.0,309.0,678.0
559,8933105,1,267,0,0.0,108.0,0.0,0.0,268,108.0,1.0,159.0,160.0,320.0
555,8933099,0,131,0,0.0,33.0,0.0,0.0,131,33.0,0.0,98.0,98.0,196.0
166,8927484,3,95,1,0.0,2.0,0.0,0.0,98,2.0,3.0,93.0,96.0,192.0
200,8927555,1,11,92,0.0,104.0,0.0,0.0,12,104.0,1.0,93.0,92.0,186.0
195,8927546,53,449,18,0.0,428.0,0.0,0.0,502,428.0,53.0,21.0,74.0,148.0
621,8933229,0,93,0,0.0,27.0,0.0,0.0,93,27.0,0.0,66.0,66.0,132.0
190,8927536,0,190,68,0.0,254.0,0.0,0.0,190,254.0,0.0,64.0,64.0,128.0
538,8933074,5,58,0,0.0,5.0,0.0,0.0,63,5.0,5.0,53.0,58.0,116.0
536,8933072,2,63,0,0.0,9.0,0.0,0.0,65,9.0,2.0,54.0,56.0,112.0


In [9]:
# extract_examples(merged[merged["patentnumber"] == "9018332"]["xml"].values[0])

In [ ]:
[8969641,8926966,8951515,8940436,9023999,9011956,9018396,9213027,9108964,9084992]#examples our algorithm didnt find but freilich did 
# many dont have an overarching examples section but have example 1, example 2 etc
# check these xml, the text has examples section:9194008,9018332,9054322,9150561,9213027, 

In [10]:
# import pprint as p
# p.pprint(merged[merged.patentnumber == "9018332"]["xml"].values[0])

In [22]:
def extract_examples_start_w_word(xml_siblings):
    examples = []
    current_example = None
    in_example = False

    for tag in xml_siblings:
        if tag.name == "heading":
            text_lower = tag.text.strip().lower()
            if (
                "example" in text_lower
                or "experiment" in text_lower
                or "test" in text_lower
            ):
                in_example = True
                current_example = {
                    "number": tag.text.strip(),
                    "title": xml_siblings[xml_siblings.index(tag) + 1].text.strip(),
                    "content": [],
                }
                examples.append(current_example)
            else:
                # If we hit any other heading, stop collecting content
                in_example = False
        elif in_example and tag.name == "p" and current_example is not None:
            current_example["content"].append(tag.text.strip())

    return examples

In [27]:
import numpy as np

df_check = df_final.merge(merged,on= "patentnumber",how="left")[["patentnumber","present","past","Unknown","prophetic","nonprophetic","allprophetic","someprophetic"]]
#df_check["past"] = df_check["past"] + df_check["Unknown"]
df_check["Total_Extracted"] = df_check["past"] + df_check["present"]  #+ df_check["Unknown"]
df_check["Total_Freilich"] = df_check["prophetic"] + df_check["nonprophetic"]
df_check["prophetic_error"] = np.sqrt((df_check["prophetic"] - df_check["present"])**2)
df_check["nonprophetic_error"] = np.sqrt((df_check["nonprophetic"] - df_check["past"])**2)
df_check["Total_Mean_error"] = np.sqrt((df_check["Total_Freilich"] - df_check["Total_Extracted"])**2)
df_check["Sum_error"] = df_check["prophetic_error"] + df_check["nonprophetic_error"] + df_check["Total_Mean_error"]
print(f"Number of exact matches: {len(df_check[(df_check.prophetic_error ==0) & (df_check.nonprophetic_error ==0) & (df_check.Total_Mean_error ==0)])} out of {len(df_check)} , Percentage: {len(df_check[(df_check.prophetic_error ==0) & (df_check.nonprophetic_error ==0) & (df_check.Total_Mean_error ==0)])/len(df_check)*100}")
print(f"Number of exact num of patent extracted: {len(df_check[(df_check.Total_Extracted == df_check.Total_Freilich)])} out of {len(df_check)}")
print(f"Avg Total error: {df_check['Total_Mean_error'].mean()}, num of corrects:  {df_check[df_check['Total_Mean_error'] == 0].shape[0]}")
print(f"Avg Total prophetic error: {df_check['prophetic_error'].mean()}, num of corrects: {df_check[df_check['prophetic_error'] == 0].shape[0]}")
print(f"Avg Total nonprophetic error: {df_check['nonprophetic_error'].mean()}, num of corrects: {df_check[df_check['nonprophetic_error'] == 0].shape[0]}")
print(f"{df_check['Sum_error'].sum()}")
df_check.sort_values("Total_Mean_error",ascending=False).head(20)



#With clean_text and tense updated description for first 2000
# Number of exact matches: 626 out of 2000 , Percentage: 31.3
# Number of exact num of patent extracted: 1254 out of 2000
# Avg Total error: 2.1615, num of corrects:  1254
# Avg Total prophetic error: 1.4895, num of corrects: 1041
# Avg Total nonprophetic error: 3.5455, num of corrects: 733
# 14393.0


# not extracting reference examples
# Number of exact matches: 641 out of 2000 , Percentage: 32.05
# Number of exact num of patent extracted: 1274 out of 2000
# Avg Total error: 1.717, num of corrects:  1274
# Avg Total prophetic error: 1.483, num of corrects: 1045
# Avg Total nonprophetic error: 3.126, num of corrects: 750
# 12652.0


# not extracting reference and preparation examples
# Number of exact matches: 650 out of 2000 , Percentage: 32.5
# Number of exact num of patent extracted: 1293 out of 2000
# Avg Total error: 1.561,f num of corrects:  1293
# Avg Total prophetic error: 1.4595, num of corrects: 1049
# Avg Total nonprophetic error: 3.028, num of corrects: 751
# 12097.0


# I have enhanced the algorithm with a different combination of the 
# techniques mentioned in previous email, now our algorithm finds examples in 75.6% 
# of janets dataset for 2015 which is a 25% increase compared to previous algorthmic 
# approach.( from the 25081 patents in janets for 2015 we found 18977 patents with examples)
#After removing all the unknowsn: we have 1227 out of 2000 (61%) patents with the number of
#  examples as janet. The total error calculated as before is now at 17778. which is 
# substantially bigger than before
#in 86 cases we have less than freilich, in 687 cases we have more than freilich (e.g refrence,test,
# production examples). we extracted the same number of prophetics as janets in 50% (613/1227 ) of the patents 
# where we extracted the same number of total patents as freilich.( i rerun the code for 5000 
# patents and got 79.5% patents with the same number of prophetics as frelich, and got 61% (3058/5000) with the 
# same total number of patents as janets). I would like to mention, now we are extracting the examples from patents that dont have experiments
#section aswell, therefore tha i checked its effect on our resuts accuracy. The previous aproach (only extracting from experiments section)
# resulted in an overall erro (Total num of patents avg diff + total num of prophetics 
# avg diff + total num of nonprophetics avg diff) of 7.2, while our new approach resulted in 8.88.


# Number of exact matches: 613 out of 2000 , Percentage: 30.65
# Number of exact num of patent extracted: 1227 out of 2000
# Avg Total error: 3.0625, num of corrects:  1227
# Avg Total prophetic error: 1.8045, num of corrects: 961
# Avg Total nonprophetic error: 4.022, num of corrects: 706
# 17778.0


# Number of exact matches: 1550 out of 5000 , Percentage: 31.0
# Number of exact num of patent extracted: 3058 out of 5000
# Avg Total error: 3.0708, num of corrects:  3058
# Avg Total prophetic error: 1.7164, num of corrects: 2433
# Avg Total nonprophetic error: 3.9832, num of corrects: 1797
# 43852.0

# threshhold = 0
# Number of exact matches: 237 out of 772 , Percentage: 30.699481865284973
# Number of exact num of patent extracted: 473 out of 772
# Avg Total error: 3.05699481865285, num of corrects:  473
# Avg Total prophetic error: 1.6735751295336787, num of corrects: 383
# Avg Total nonprophetic error: 3.9274611398963732, num of corrects: 269
# 6684.0


# threshhold = 50
# Number of exact matches: 239 out of 772 , Percentage: 30.958549222797927
# Number of exact num of patent extracted: 474 out of 772
# Avg Total error: 3.051813471502591, num of corrects:  474
# Avg Total prophetic error: 1.6593264248704662, num of corrects: 385
# Avg Total nonprophetic error: 3.928756476683938, num of corrects: 269
# 6670.0

# threshhold = 100
# Number of exact matches: 238 out of 772 , Percentage: 30.82901554404145
# Number of exact num of patent extracted: 478 out of 772
# Avg Total error: 3.045336787564767, num of corrects:  478
# Avg Total prophetic error: 1.6398963730569949, num of corrects: 387
# Avg Total nonprophetic error: 3.91839378238342, num of corrects: 268
# 6642.0

# threshhold = 150
# Number of exact matches: 239 out of 772 , Percentage: 30.958549222797927
# Number of exact num of patent extracted: 472 out of 772
# Avg Total error: 3.038860103626943, num of corrects:  472
# Avg Total prophetic error: 1.6204663212435233, num of corrects: 392
# Avg Total nonprophetic error: 3.905440414507772, num of corrects: 266
# 6612.0


# threshhold = 200
# Number of exact matches: 238 out of 772 , Percentage: 30.82901554404145
# Number of exact num of patent extracted: 466 out of 772
# Avg Total error: 3.022020725388601, num of corrects:  466
# Avg Total prophetic error: 1.6139896373056994, num of corrects: 396
# Avg Total nonprophetic error: 3.866580310880829, num of corrects: 264
# 6564.0

Number of exact matches: 239 out of 772 , Percentage: 30.958549222797927
Number of exact num of patent extracted: 472 out of 772
Avg Total error: 3.038860103626943, num of corrects:  472
Avg Total prophetic error: 1.6204663212435233, num of corrects: 392
Avg Total nonprophetic error: 3.905440414507772, num of corrects: 266
6612.0


,patentnumber,present,past,Unknown,prophetic,nonprophetic,allprophetic,someprophetic,Total_Extracted,Total_Freilich,prophetic_error,nonprophetic_error,Total_Mean_error,Sum_error
277,8927710,29,204,169,0.0,564.0,0.0,0.0,233,564.0,29.0,360.0,331.0,720.0
559,8933105,1,267,0,0.0,108.0,0.0,0.0,268,108.0,1.0,159.0,160.0,320.0
555,8933099,0,131,0,0.0,33.0,0.0,0.0,131,33.0,0.0,98.0,98.0,196.0
166,8927484,3,95,0,0.0,2.0,0.0,0.0,98,2.0,3.0,93.0,96.0,192.0
200,8927555,1,11,92,0.0,104.0,0.0,0.0,12,104.0,1.0,93.0,92.0,186.0
621,8933229,0,93,0,0.0,27.0,0.0,0.0,93,27.0,0.0,66.0,66.0,132.0
190,8927536,0,190,1,0.0,254.0,0.0,0.0,190,254.0,0.0,64.0,64.0,128.0
195,8927546,50,442,0,0.0,428.0,0.0,0.0,492,428.0,50.0,14.0,64.0,128.0
538,8933074,5,58,0,0.0,5.0,0.0,0.0,63,5.0,5.0,53.0,58.0,116.0
536,8933072,2,63,0,0.0,9.0,0.0,0.0,65,9.0,2.0,54.0,56.0,112.0


In [15]:
df_check.Unknown[:2000].sum()

2780

In [33]:
1254/2000

0.627

In [ ]:

len(df_check[df_check.Total_Freilich == df_check.Total_Extracted]["Total_Mean_error"])

0

# ERRORS IN FREILICH DATASET:
patentnumber 8957069: in freilich 108.0	nonproph and 0 prophetic but there are many more examples than 108, our algorithm: 267 nonprophetic and 1 prophetic and 3 unknown
patent number 8946443:we extracted 271	freilich: 108, this patent has refrtence examples, are we extracting these or not? 
patent number 8952010: same issue do we take refrence examples? 
patent 8933099: there are far more than 33 examples in this patent but freilich says 33, our algorithm found 131
what to do with Preparation examples?? e.g 8962612 in freilich has 6 we got 70.

# improvement for our algorithm
1. remove duplicates somehow, e.g in patent number 08987242, there some examples are extracted twice just because the name e.g example aa137 appears multiple times.
2. better classification of the patents. e.g patent 8987295 has 442, we extract the same number as frelich but we clasify differently.
3. 

In [92]:
check_tense_nltk("""9-(4-(1-Aminoethyl)phenyl)-6-chloro-8-hydroxythieno[2,3-c]quinolin-4(5H)-o- ne Hydrochloride
(573) ##STR00933##
(574) Following General Procedure F, tert-butyl 1-(4-(6-chloro-8-methoxy-4-oxo-4,5-dihydrothieno[2,3-c]quinolin-9-yl)phen- yl)ethylcarbamate (50 mg, 0.10 mmol) was treated with BBr.sub.3 (1.0 M in CH.sub.2Cl.sub.2, 2 mL, 2 mmol) to afford the desired product (21 mg, 58%) as a light yellow solid (21 mg, 58%): .sup.1H NMR (500 MHz, CD.sub.4OD) .delta. 7.65 (dt, J=5.2, 3.4 Hz, 2H), 7.59 (d, J=5.4 Hz, 1H), 7.41 (dt, J=4.0, 2.6 Hz, 2H), 7.30 (s, 1H), 6.07 (d, J=: 5.4 Hz, 1H), 4.62 (q, J=6.8 Hz, 1H), 1.76 (d, J=6.9 Hz, 3H); ESI MS m/z 371 [C.sub.19H.sub.15ClN.sub.1O.sub.2S+H].sup.+; HPLC 97.8% (AUC), t.sub.R=9.72 min.""")

'past'

In [83]:
df_check[df_check.Unknown > 0]

,patentnumber,present,past,Unknown,prophetic,nonprophetic,allprophetic,someprophetic,Total_Extracted,Total_Freilich,prophetic_error,nonprophetic_error,Total_Mean_error
12,8946423,1,9,1,0.0,9.0,0.0,0.0,11,9.0,1.0,0.0,2.0
16,8946430,4,38,11,0.0,53.0,0.0,0.0,53,53.0,4.0,15.0,0.0
17,8946434,3,23,3,3.0,23.0,0.0,1.0,29,26.0,0.0,0.0,3.0
18,8946437,8,19,34,0.0,40.0,0.0,0.0,61,40.0,8.0,21.0,21.0


In [35]:
merged[merged.patentnumber == "8926083"][cols]

,patentnumber,xml,prophetic,nonprophetic,allprophetic,someprophetic
8,8926083,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT...",0.0,49.0,0.0,0.0
